In [32]:
import torch
import torchaudio
import soundfile as sf
import numpy as np
from transformers import pipeline
from speechbrain.pretrained import EncoderClassifier

def extract_speaker_embedding(audio_path):
    try:
        # Load a pre-trained speaker embedding model
        classifier = EncoderClassifier.from_hparams(
            source="speechbrain/spkrec-xvect-voxceleb",
            savedir="tmp"
        )

        # Load audio file
        signal, fs = torchaudio.load(audio_path)

        # Resample if needed (most models prefer 16 kHz)
        if fs != 16000:
            resample_transform = torchaudio.transforms.Resample(
                orig_freq=fs,
                new_freq=16000
            )
            signal = resample_transform(signal)

        # Ensure mono audio
        if signal.shape[0] > 1:
            signal = signal.mean(dim=0, keepdim=True)

        # Ensure the signal is 2D (batch, samples)
        if signal.ndim == 1:
            signal = signal.unsqueeze(0)

        # Generate speaker embedding
        embeddings = classifier.encode_batch(signal)
        speaker_embedding = embeddings.mean(dim=1)

        # Ensure the embedding is a tensor
        if not torch.is_tensor(speaker_embedding):
            speaker_embedding = torch.tensor(speaker_embedding)

        # Ensure the embedding has the right shape (batch, embedding_dim)
        if speaker_embedding.ndim == 1:
            speaker_embedding = speaker_embedding.unsqueeze(0)

        return speaker_embedding

    except Exception as e:
        print(f"Error extracting speaker embedding: {e}")
        return None

def synthesize_speech(
    text_list,
    speaker_embedding,
    output_path='synthesized_speech.wav',
    model_name="microsoft/speecht5_tts"
):
    try:
        # Determine device
        device = 0 if torch.cuda.is_available() else -1

        # Initialize TTS pipeline
        synthesiser = pipeline(
            "text-to-speech",
            model=model_name,
            device=device
        )

        # Ensure speaker embedding matches batch size
        if speaker_embedding.shape[0] != len(text_list):
            # If single embedding, repeat to match text batch size
            speaker_embedding = speaker_embedding.repeat(len(text_list), 1)

        # Synthesize speech
        speech = synthesiser(
            text_list,
            forward_params={"speaker_embeddings": speaker_embedding}
        )

        # Save synthesized speech
        sf.write(
            output_path,
            speech[0]["audio"],
            samplerate=speech[0]["sampling_rate"]
        )

        return output_path

    except Exception as e:
        print(f"Error synthesizing speech: {e}")
        print(f"Speaker embedding shape: {speaker_embedding.shape}")
        print(f"Number of texts: {len(text_list)}")
        return None

def main():
    # Path to your reference voice audio
    reference_audio_path = "/content/drive/MyDrive/imran.wav"

    # Extract speaker embedding
    speaker_embedding = extract_speaker_embedding(reference_audio_path)

    if speaker_embedding is not None:
        # Text to synthesize
        texts_to_speak = [
            "I am Ahmad, recently graduated from FAST University Islamabad.",
            "This is a demonstration of voice cloning technology."
        ]

        # Synthesize speech
        output_file = synthesize_speech(
            texts_to_speak,
            speaker_embedding
        )

        if output_file:
            print(f"Speech synthesis complete! Audio saved to: {output_file}")
    else:
        print("Failed to extract speaker embedding.")

if __name__ == "__main__":
    main()

Device set to use cuda:0


Error synthesizing speech: The first dimension of speaker_embeddings must be either 1 or the same as batch_size.
Speaker embedding shape: torch.Size([2, 512])
Number of texts: 2


In [33]:
import torch
import torchaudio
import soundfile as sf
import numpy as np
from transformers import pipeline
from speechbrain.pretrained import EncoderClassifier

def extract_speaker_embedding(audio_path):
    try:
        # Load a pre-trained speaker embedding model
        classifier = EncoderClassifier.from_hparams(
            source="speechbrain/spkrec-xvect-voxceleb",
            savedir="tmp"
        )

        # Load audio file
        signal, fs = torchaudio.load(audio_path)

        # Resample if needed (most models prefer 16 kHz)
        if fs != 16000:
            resample_transform = torchaudio.transforms.Resample(
                orig_freq=fs,
                new_freq=16000
            )
            signal = resample_transform(signal)

        # Ensure mono audio
        if signal.shape[0] > 1:
            signal = signal.mean(dim=0, keepdim=True)

        # Ensure the signal is 2D (batch, samples)
        if signal.ndim == 1:
            signal = signal.unsqueeze(0)

        # Generate speaker embedding
        embeddings = classifier.encode_batch(signal)
        speaker_embedding = embeddings.mean(dim=1)

        # Reduce to a single embedding
        if speaker_embedding.shape[0] > 1:
            speaker_embedding = speaker_embedding.mean(dim=0, keepdim=True)

        return speaker_embedding

    except Exception as e:
        print(f"Error extracting speaker embedding: {e}")
        return None

def synthesize_speech(
    text_list,
    speaker_embedding,
    output_path='synthesized_speech.wav',
    model_name="microsoft/speecht5_tts"
):
    try:
        # Determine device
        device = 0 if torch.cuda.is_available() else -1

        # Initialize TTS pipeline
        synthesiser = pipeline(
            "text-to-speech",
            model=model_name,
            device=device
        )

        # Ensure speaker embedding is a single vector
        if speaker_embedding.shape[0] > 1:
            speaker_embedding = speaker_embedding.mean(dim=0, keepdim=True)

        # Repeat embedding to match text batch size
        speaker_embedding = speaker_embedding.repeat(len(text_list), 1)

        # Print debug information
        print("Final speaker embedding shape:", speaker_embedding.shape)
        print("Number of texts:", len(text_list))

        # Synthesize speech
        speech = synthesiser(
            text_list,
            forward_params={"speaker_embeddings": speaker_embedding}
        )

        # Save synthesized speech
        sf.write(
            output_path,
            speech[0]["audio"],
            samplerate=speech[0]["sampling_rate"]
        )

        return output_path

    except Exception as e:
        print(f"Error synthesizing speech: {e}")
        return None

def main():
    # Path to your reference voice audio
    reference_audio_path = "/content/drive/MyDrive/imran.wav"

    # Extract speaker embedding
    speaker_embedding = extract_speaker_embedding(reference_audio_path)

    if speaker_embedding is not None:
        # Print embedding shape for debugging
        print("Initial speaker embedding shape:", speaker_embedding.shape)

        # Text to synthesize
        texts_to_speak = [
            "I am Ahmad, recently graduated from FAST University Islamabad.",
            "This is a demonstration of voice cloning technology."
        ]

        # Synthesize speech
        output_file = synthesize_speech(
            texts_to_speak,
            speaker_embedding
        )

        if output_file:
            print(f"Speech synthesis complete! Audio saved to: {output_file}")
    else:
        print("Failed to extract speaker embedding.")

if __name__ == "__main__":
    main()

Initial speaker embedding shape: torch.Size([1, 512])


Device set to use cuda:0


Final speaker embedding shape: torch.Size([2, 512])
Number of texts: 2
Error synthesizing speech: The first dimension of speaker_embeddings must be either 1 or the same as batch_size.


In [4]:
!pip install speechbrain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 24.5 MB/s eta 0:00:00


In [3]:
import torch
import torchaudio
import soundfile as sf
import numpy as np
from transformers import pipeline
from speechbrain.pretrained import EncoderClassifier

def extract_speaker_embedding(audio_path):
    try:
        # Load a pre-trained speaker embedding model
        classifier = EncoderClassifier.from_hparams(
            source="speechbrain/spkrec-xvect-voxceleb",
            savedir="tmp"
        )

        # Load audio file
        signal, fs = torchaudio.load(audio_path)

        # Resample if needed (most models prefer 16 kHz)
        if fs != 16000:
            resample_transform = torchaudio.transforms.Resample(
                orig_freq=fs,
                new_freq=16000
            )
            signal = resample_transform(signal)

        # Ensure mono audio
        if signal.shape[0] > 1:
            signal = signal.mean(dim=0, keepdim=True)

        # Ensure the signal is 2D (batch, samples)
        if signal.ndim == 1:
            signal = signal.unsqueeze(0)

        # Generate speaker embedding
        embeddings = classifier.encode_batch(signal)
        speaker_embedding = embeddings.mean(dim=1)

        # Squeeze to remove unnecessary dimensions
        speaker_embedding = speaker_embedding.squeeze()

        return speaker_embedding

    except Exception as e:
        print(f"Error extracting speaker embedding: {e}")
        return None

def synthesize_speech(
    text_list,
    speaker_embedding,
    output_path='synthesized_speech.wav',
    model_name="microsoft/speecht5_tts"
):
    try:
        # Determine device
        device = 0 if torch.cuda.is_available() else -1

        # Initialize TTS pipeline
        synthesiser = pipeline(
            "text-to-speech",
            model=model_name,
            device=device
        )

        # Ensure speaker embedding is a 1D tensor
        if speaker_embedding.ndim == 1:
            # Add a batch dimension
            speaker_embedding = speaker_embedding.unsqueeze(0)

        print("Initial speaker embedding shape:", speaker_embedding.shape)
        print("Number of texts:", len(text_list))

        # Synthesize speech
        speech = synthesiser(
            text_list,
            forward_params={"speaker_embeddings": speaker_embedding}
        )

        # Save synthesized speech
        sf.write(
            output_path,
            speech[0]["audio"],
            samplerate=speech[0]["sampling_rate"]
        )

        return output_path

    except Exception as e:
        print(f"Error synthesizing speech: {e}")
        return None

def main():
    # Path to your reference voice audio
    reference_audio_path = "/content/drive/MyDrive/me.wav"

    # Extract speaker embedding
    speaker_embedding = extract_speaker_embedding(reference_audio_path)

    if speaker_embedding is not None:
        # Text to synthesize
        texts_to_speak = [
            "My name is Mohsin, and I have completed sixteen years of education.eering], I am eager to apply my expertise in CS",
            "This is a demonstration of voice cloning technology."
        ]

        # Synthesize speech
        output_file = synthesize_speech(
            texts_to_speak,
            speaker_embedding
        )

        if output_file:
            print(f"Speech synthesis complete! Audio saved to: {output_file}")
    else:
        print("Failed to extract speaker embedding.")

if __name__ == "__main__":
    main()

<ipython-input-3-193c528c93e2>:6: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier


hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

Initial speaker embedding shape: torch.Size([1, 512])
Number of texts: 2
Speech synthesis complete! Audio saved to: synthesized_speech.wav


In [3]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [5]:
import gradio as gr
import torch
import torchaudio
import soundfile as sf
from transformers import pipeline
from speechbrain.pretrained import EncoderClassifier

def extract_speaker_embedding(audio_path):
    try:
        classifier = EncoderClassifier.from_hparams(
            source="speechbrain/spkrec-xvect-voxceleb",
            savedir="tmp"
        )
        signal, fs = torchaudio.load(audio_path)
        if fs != 16000:
            resample_transform = torchaudio.transforms.Resample(orig_freq=fs, new_freq=16000)
            signal = resample_transform(signal)
        if signal.shape[0] > 1:
            signal = signal.mean(dim=0, keepdim=True)
        if signal.ndim == 1:
            signal = signal.unsqueeze(0)
        embeddings = classifier.encode_batch(signal)
        speaker_embedding = embeddings.mean(dim=1).squeeze()
        return speaker_embedding
    except Exception as e:
        return f"Error extracting speaker embedding: {e}"

def synthesize_speech(audio_file, texts):
    try:
        # Extract speaker embedding
        speaker_embedding = extract_speaker_embedding(audio_file)
        if isinstance(speaker_embedding, str):  # Error during embedding extraction
            return speaker_embedding

        # Set up TTS pipeline
        device = 0 if torch.cuda.is_available() else -1
        synthesiser = pipeline("text-to-speech", model="microsoft/speecht5_tts", device=device)

        # Ensure embedding format
        if speaker_embedding.ndim == 1:
            speaker_embedding = speaker_embedding.unsqueeze(0)

        # Generate speech
        text_list = texts.split("\n")
        speech = synthesiser(
            text_list,
            forward_params={"speaker_embeddings": speaker_embedding}
        )

        # Save and return synthesized audio
        output_path = "synthesized_speech.wav"
        sf.write(output_path, speech[0]["audio"], samplerate=speech[0]["sampling_rate"])
        return output_path
    except Exception as e:
        return f"Error during speech synthesis: {e}"

def generate_audio(audio_file, text_input):
    output = synthesize_speech(audio_file, text_input)
    if output.endswith(".wav"):
        return output
    else:
        return f"Error: {output}"

# Gradio Interface
with gr.Blocks(css=".container {background-color: #f0f8ff;} .button {background-color: #00bfff; color: white; padding: 10px; border-radius: 5px;}") as demo:
    gr.Markdown("""
    # 🎙️ **Voice Cloning Application**
    Experience state-of-the-art voice cloning using deep learning!
    """, elem_classes="container")

    gr.Markdown("""
    ## 📂 **Step 1: Upload Reference Audio**
    Provide a short audio clip of the speaker whose voice you want to clone.
    """)
    audio_file = gr.Audio(label="Upload Reference Audio", type="filepath")

    gr.Markdown("""
    ## ✍️ **Step 2: Enter Text**
    Type the sentences you want to generate speech for. You can enter multiple sentences, each on a new line.
    """)
    text_input = gr.Textbox(
        label="Enter Text (separate multiple sentences with a new line)",
        placeholder="Type your sentences here..."
    )

    gr.Markdown("---")  # Adds a horizontal line separator

    gr.Markdown("""
    ## 🎧 **Step 3: Generate Cloned Speech**
    Click the button below to synthesize speech in the cloned voice.
    """)
    output_audio = gr.Audio(label="Synthesized Audio")

    synthesize_button = gr.Button("🎤 Synthesize Speech", elem_classes="button")
    synthesize_button.click(generate_audio, inputs=[audio_file, text_input], outputs=output_audio)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3325ea82293e9f2c7a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [2]:
!pip install openai-whisper
!pip install torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803320 sha256=fbba70aa402ef6b6e95ec448c10ee45e9e24ecdb4538d11c197c83fffe3a6459
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [13]:
import whisper


model = whisper.load_model("base")


def transcribe_audio(audio_path):
    # Transcribe the audio using the model
    result = model.transcribe(audio_path)
    return result['text']


audio_path = '/content/drive/MyDrive/me.wav'


transcribed_text = transcribe_audio(audio_path)
print("Transcribed Text:\n", transcribed_text)


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 94.3MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcribed Text:
  Voice Cloning referred to the purchase of creating synthetic version of someone's voice dad sound alike original speaker. This technology uses deep learning techniques, particularly speech synthesis and voice conversion models to replicate the unique reactive of per person voice, such as tone, pitch, accent. Voice Cloning typically involves the holding step. Data collection pre-processing, motor training, voice synthesis. Then the primary application of voice cloning include personalized, virtual assistant, content creation, voice overs, and more controversially, and synthetic media. Ethical considerations are important as voice cloning can be misused for deceptive purposes.


In [6]:
import torch
import torchaudio
import soundfile as sf
from transformers import pipeline
from speechbrain.pretrained import EncoderClassifier
import whisper

# Function to extract speaker embedding
def extract_speaker_embedding(audio_path):
    try:
        # Load a pre-trained speaker embedding model
        classifier = EncoderClassifier.from_hparams(
            source="speechbrain/spkrec-xvect-voxceleb",
            savedir="tmp"
        )

        # Load audio file
        signal, fs = torchaudio.load(audio_path)

        # Resample if needed (most models prefer 16 kHz)
        if fs != 16000:
            resample_transform = torchaudio.transforms.Resample(
                orig_freq=fs,
                new_freq=16000
            )
            signal = resample_transform(signal)

        # Ensure mono audio
        if signal.shape[0] > 1:
            signal = signal.mean(dim=0, keepdim=True)

        # Ensure the signal is 2D (batch, samples)
        if signal.ndim == 1:
            signal = signal.unsqueeze(0)

        # Generate speaker embedding
        embeddings = classifier.encode_batch(signal)
        speaker_embedding = embeddings.mean(dim=1)

        # Squeeze to remove unnecessary dimensions
        speaker_embedding = speaker_embedding.squeeze()

        return speaker_embedding

    except Exception as e:
        print(f"Error extracting speaker embedding: {e}")
        return None

# Function to transcribe audio to text
def transcribe_audio(audio_path):
    try:
        # Load the pretrained Whisper model
        model = whisper.load_model("base")
        # Transcribe the audio using the model
        result = model.transcribe(audio_path)
        return result['text']
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

# Function to synthesize speech
def synthesize_speech(
    text,
    speaker_embedding,
    output_path='synthesized_speech.wav',
    model_name="microsoft/speecht5_tts"
):
    try:
        # Determine device
        device = 0 if torch.cuda.is_available() else -1

        # Initialize TTS pipeline
        synthesiser = pipeline(
            "text-to-speech",
            model=model_name,
            device=device
        )

        # Ensure speaker embedding is a 1D tensor
        if speaker_embedding.ndim == 1:
            speaker_embedding = speaker_embedding.unsqueeze(0)

        # Synthesize speech
        speech = synthesiser(
            [text],
            forward_params={"speaker_embeddings": speaker_embedding}
        )

        # Save synthesized speech
        sf.write(
            output_path,
            speech[0]["audio"],
            samplerate=speech[0]["sampling_rate"]
        )

        return output_path

    except Exception as e:
        print(f"Error synthesizing speech: {e}")
        return None

# Main function
def main():
    # Paths to the reference and input audio files
    reference_audio_path = "/content/drive/MyDrive/imran.wav"  # Path to target voice
    input_audio_path = "/content/drive/MyDrive/me.wav"  # Path to input audio for transcription

    # Extract speaker embedding
    speaker_embedding = extract_speaker_embedding(reference_audio_path)

    if speaker_embedding is not None:
        # Transcribe audio
        transcribed_text = transcribe_audio(input_audio_path)

        if transcribed_text:
            # Use only the first sentence of the transcribed text
            first_sentence = transcribed_text.split(".")[0].strip()

            print(f"First Sentence from Transcription: {first_sentence}")

            # Synthesize speech
            output_file = synthesize_speech(
                first_sentence,
                speaker_embedding
            )

            if output_file:
                print(f"Speech synthesis complete! Audio saved to: {output_file}")
        else:
            print("Failed to transcribe audio.")
    else:
        print("Failed to extract speaker embedding.")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

First Sentence from Transcription: Voice Cloning referred to the purchase of creating synthetic version of someone's voice dad sound alike original speaker
Speech synthesis complete! Audio saved to: synthesized_speech.wav


In [7]:
import gradio as gr
import torch
import torchaudio
import soundfile as sf
from transformers import pipeline
from speechbrain.pretrained import EncoderClassifier
import whisper

# Function to extract speaker embedding
def extract_speaker_embedding(audio_path):
    try:
        # Load a pre-trained speaker embedding model
        classifier = EncoderClassifier.from_hparams(
            source="speechbrain/spkrec-xvect-voxceleb",
            savedir="tmp"
        )

        # Load audio file
        signal, fs = torchaudio.load(audio_path)

        # Resample if needed (most models prefer 16 kHz)
        if fs != 16000:
            resample_transform = torchaudio.transforms.Resample(
                orig_freq=fs,
                new_freq=16000
            )
            signal = resample_transform(signal)

        # Ensure mono audio
        if signal.shape[0] > 1:
            signal = signal.mean(dim=0, keepdim=True)

        # Ensure the signal is 2D (batch, samples)
        if signal.ndim == 1:
            signal = signal.unsqueeze(0)

        # Generate speaker embedding
        embeddings = classifier.encode_batch(signal)
        speaker_embedding = embeddings.mean(dim=1)

        # Squeeze to remove unnecessary dimensions
        speaker_embedding = speaker_embedding.squeeze()

        return speaker_embedding

    except Exception as e:
        return str(f"Error extracting speaker embedding: {e}")

# Function to transcribe audio to text
def transcribe_audio(audio_path):
    try:
        # Load the pretrained Whisper model
        model = whisper.load_model("base")
        # Transcribe the audio using the model
        result = model.transcribe(audio_path)
        return result['text']
    except Exception as e:
        return str(f"Error transcribing audio: {e}")

# Function to synthesize speech
def synthesize_speech(
    text,
    speaker_embedding,
    output_path='synthesized_speech.wav',
    model_name="microsoft/speecht5_tts"
):
    try:
        # Determine device
        device = 0 if torch.cuda.is_available() else -1

        # Initialize TTS pipeline
        synthesiser = pipeline(
            "text-to-speech",
            model=model_name,
            device=device
        )

        # Ensure speaker embedding is a 1D tensor
        if speaker_embedding.ndim == 1:
            speaker_embedding = speaker_embedding.unsqueeze(0)

        # Synthesize speech
        speech = synthesiser(
            [text],
            forward_params={"speaker_embeddings": speaker_embedding}
        )

        # Save synthesized speech
        sf.write(
            output_path,
            speech[0]["audio"],
            samplerate=speech[0]["sampling_rate"]
        )

        return output_path

    except Exception as e:
        return str(f"Error synthesizing speech: {e}")
# Gradio function for the pipeline
def process_audio(reference_audio_path, input_audio_path):
    try:
        # Extract speaker embedding
        speaker_embedding = extract_speaker_embedding(reference_audio_path)
        if isinstance(speaker_embedding, str):  # Check for errors
            return speaker_embedding, None

        # Transcribe input audio
        transcribed_text = transcribe_audio(input_audio_path)
        if isinstance(transcribed_text, str) and transcribed_text.startswith("Error"):
            return transcribed_text, None

        # Use the first sentence of the transcription
        first_sentence = transcribed_text.split(".")[0].strip()

        # Synthesize speech
        output_file = synthesize_speech(
            first_sentence,
            speaker_embedding,
            output_path="synthesized_output.wav"
        )
        if isinstance(output_file, str) and output_file.startswith("Error"):
            return output_file, None

        return first_sentence, output_file

    except Exception as e:
        return str(f"Unexpected error: {e}"), None


# Create Gradio interface
interface = gr.Interface(
    fn=process_audio,
    inputs=[
        gr.Audio(label="Reference Audio (Target Voice)", type="filepath"),
        gr.Audio(label="Input Audio (To Transcribe)", type="filepath")
    ],
    outputs=[
        gr.Textbox(label="Transcribed First Sentence"),
        gr.Audio(label="Synthesized Speech")
    ],
    title="Voice Conversion Pipeline",
    description="Upload a reference audio file to extract voice characteristics and an input audio file for transcription. The system will synthesize the transcribed text in the reference voice."
)

# Launch the Gradio app
if __name__ == "__main__":
    interface.launch()



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7fe55da24d802603bb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
